## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import time

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,59.81,89,83,3.85,broken clouds
1,1,Mataura,NZ,-46.1927,168.8643,59.49,88,90,1.72,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,79.86,72,64,2.28,broken clouds
3,3,Khatanga,RU,71.9667,102.5000,-13.54,90,25,8.63,scattered clouds
4,4,Tromso,NO,69.6496,18.9570,29.05,86,75,1.99,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp =float(input('What is the lowest temperature of a destination you would want to travel to? '))
max_temp =float(input('What is the highest temperature of a destination you would want to travel to? '))

What is the lowest temperature of a destination you would want to travel to? 80
What is the highest temperature of a destination you would want to travel to? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
clean_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
22,22,East London,ZA,-33.0153,27.9116,85.14,65,20,13.80,few clouds
23,23,Kavieng,PG,-2.5744,150.7967,82.31,75,79,4.79,broken clouds
25,25,Avarua,CK,-21.2078,-159.7750,80.65,89,75,14.97,light rain
28,28,Banjar,ID,-8.1900,114.9675,80.46,91,100,4.21,overcast clouds
29,29,Port Elizabeth,ZA,-33.9180,25.5701,85.10,65,20,27.63,few clouds
...,...,...,...,...,...,...,...,...,...,...
404,404,Grand Gaube,MU,-20.0064,57.6608,86.38,73,40,4.00,scattered clouds
407,407,Kattivakkam,IN,13.2167,80.3167,84.29,78,25,13.56,scattered clouds
413,413,Sao Joao Da Barra,BR,-21.6403,-41.0511,80.69,82,99,5.73,overcast clouds
421,421,Casian,PH,11.0350,119.7136,80.15,77,22,26.17,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
for index, row in clean_df.iterrows():
    country_len = len(str(row['Country']))
    
    if country_len == 3:
        clean_df = clean_df.drop(index)
        print(f'Dropped {index}')

In [7]:
clean_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
22,22,East London,ZA,-33.0153,27.9116,85.14,65,20,13.80,few clouds
23,23,Kavieng,PG,-2.5744,150.7967,82.31,75,79,4.79,broken clouds
25,25,Avarua,CK,-21.2078,-159.7750,80.65,89,75,14.97,light rain
28,28,Banjar,ID,-8.1900,114.9675,80.46,91,100,4.21,overcast clouds
29,29,Port Elizabeth,ZA,-33.9180,25.5701,85.10,65,20,27.63,few clouds
...,...,...,...,...,...,...,...,...,...,...
404,404,Grand Gaube,MU,-20.0064,57.6608,86.38,73,40,4.00,scattered clouds
407,407,Kattivakkam,IN,13.2167,80.3167,84.29,78,25,13.56,scattered clouds
413,413,Sao Joao Da Barra,BR,-21.6403,-41.0511,80.69,82,99,5.73,overcast clouds
421,421,Casian,PH,11.0350,119.7136,80.15,77,22,26.17,few clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
22,East London,ZA,85.14,few clouds,-33.0153,27.9116,
23,Kavieng,PG,82.31,broken clouds,-2.5744,150.7967,
25,Avarua,CK,80.65,light rain,-21.2078,-159.7750,
28,Banjar,ID,80.46,overcast clouds,-8.1900,114.9675,
29,Port Elizabeth,ZA,85.10,few clouds,-33.9180,25.5701,
31,Hermanus,ZA,82.90,clear sky,-34.4187,19.2345,
68,Maceio,BR,81.84,scattered clouds,-9.6658,-35.7353,
72,Triunfo,BR,85.89,overcast clouds,-29.9433,-51.7181,
74,Alice Springs,AU,85.55,broken clouds,-23.7000,133.8833,
79,Lorengau,PG,81.88,overcast clouds,-2.0226,147.2712,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

count = 0

# 6b. Iterate through the hotel DataFrame.

print('Collecting Hotel Information...')

for index,row in clean_df.iterrows():
    
    count += 1
    
    if (count%50 == 0) & (count >=50):
        print('Maximum API request limit,waiting 60 seconds...')
        time.sleep(60)

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except:
        print('Hotel Not Found, skipping...')
        
print('All Done')

Hotel Not Found, skipping...
Hotel Not Found, skipping...
Maximum API request limit,waiting 60 seconds...
Hotel Not Found, skipping...
Hotel Not Found, skipping...
All Done


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [15]:
for index, row in hotel_df.iterrows():
    hotel_name = len(row['Hotel Name'])
    
    if hotel_name == 0:
        hotel_df = hotel_df.drop(index)

In [17]:
clean_hotel_df = hotel_df.copy()

In [18]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>and<dd>{Max Temp} °F</dd> 
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
hotel_marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(hotel_marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
### SAVE FIG 